In [ ]:
import optuna

import optuna.distributions as distributions

In [ ]:
def objective(trial):
    x = trial.suggest_int('x', 0, 10)
    y = trial.suggest_uniform('y', 0, 10)
    z = trial.suggest_categorical('z',[0,1])
    w = trial.suggest_discrete_uniform('w', 0.1, 1.0, 0.1)
    return x ** 2 + (y - 5)** 2 + z + (w - 0.2) ** 2
 
 
study = optuna.create_study()
study.optimize(objective, n_trials=50, n_jobs=1)

In [ ]:
#1. modify distributions
modify_distributions = {
    "y": optuna.distributions.UniformDistribution(4,6),
    "z": optuna.distributions.CategoricalDistribution([0]),
    "w": optuna.distributions.DiscreteUniformDistribution(-2.0,2.0,0.2),
}

def new_objective(trial):
    x = trial.suggest_int('x', 0, 10)
    y = trial.suggest_uniform('y', 4, 6)
    z = trial.suggest_categorical('z',[0])
    w = trial.suggest_discrete_uniform('w', -2.0, 2.0, 0.2)
 
    return x ** 2 + (y - 5)** 2 + z + (w - 0.2) ** 2

In [ ]:
# modify distributions (functions)
def _check_param(
    params,  # type: Dict[str, Any]
    modify_distributions,  # type: Dict[str, BaseDistribution]
):
    # type: (...) -> bool
    for param_name, modify_distribution in modify_distributions.items():
        if modify_distribution.isinclude(params[param_name]) is False:
            return False
    return True

def modify_distributions_map(
    modify_distributions,  # type: Dict[str, BaseDistribution]
):
    # type: (...) -> Callable
    modify_parameters = modify_distributions.keys()
    modify_distributions_done = 0

    def get_modify_distributions(trial):
        # type: (FrozenTrial) -> None
        nonlocal modify_distributions, modify_parameters, modify_distributions_done
        parameters = trial.params.keys()
        distributions = trial.distributions
        add_parameters = set(modify_parameters) - set(parameters)
        if len(add_parameters) != 0:
            raise ValueError(
                "Parameter {} is increased in new distributions".format(add_parameters)
            )
        old_parameters = set(parameters) - set(modify_parameters)
        old_distributions = {parameter: distributions[parameter] for parameter in old_parameters}
        modify_distributions.update(old_distributions)
        modify_distributions_done = 1

    def converter(trial):
        # type: (FrozenTrial) -> Optional[FrozenTrial]
        nonlocal modify_parameters, modify_distributions, modify_distributions_done
        if modify_distributions_done == 0:
            get_modify_distributions(trial)

        if _check_param(trial.params, modify_distributions) is False:
            return None

        new_trial = optuna.trial.create_trial(
            state=trial.state,
            value=trial.value,
            params=trial.params,
            distributions=modify_distributions,
            user_attrs=trial.user_attrs,
            system_attrs=trial.system_attrs,
            intermediate_values=trial.intermediate_values,
        )
        return new_trial

    return converter
        

In [ ]:
# modify distributions (execute)
converter = modify_distributions_map(modify_distributions)

new_study = optuna.create_study()
new_study.add_trials(map(lambda t: converter(t), study.trials))

new_study.optimize(new_objective, n_trials=20, n_jobs=1)

In [ ]:
#2. add parameters
add_distributions_params = {
    "add_param1": [optuna.distributions.UniformDistribution(-2,2),0],
    "add_param2": [optuna.distributions.CategoricalDistribution([False,True]),False],
}


def new_objective(trial):
    x = trial.suggest_int('x', 0, 10)
    y = trial.suggest_uniform('y', 4, 6)
    z = trial.suggest_categorical('z',[0])
    w = trial.suggest_discrete_uniform('w', -2.0, 2.0, 0.2)
    add_param1 = trial.suggest_uniform('add_param1', -2, 2)
    add_param2 = trial.suggest_categorical('add_param2',[False, True])
 
    if add_param2 is True:
        return x ** 2 + (y - 5)** 2 + z + (w - 0.2) ** 2 + add_param1
    return x ** 2 + (y - 5)** 2 + z + (w - 0.2) ** 2

In [ ]:
# add parameters (utility)
def add_distributions_params_map(
    add_distributions_params,  # type: Dict[str, List[Any]]
):
    # type: (...) -> Callable
    add_parameters = add_distributions_params.keys()
    add_distributions = {
        parameter: add_distributions_params[parameter][0] for parameter in add_parameters
    }
    add_params = {
        parameter: add_distributions_params[parameter][1] for parameter in add_parameters
    }

    def converter(trial):
        # type: (FrozenTrial) -> Optional[FrozenTrial]
        nonlocal add_distributions, add_params
        trial.params.update(add_params)
        trial.distributions.update(add_distributions)
        new_trial = optuna.trial.create_trial(
            state=trial.state,
            value=trial.value,
            params=trial.params,
            distributions=trial.distributions,
            user_attrs=trial.user_attrs,
            system_attrs=trial.system_attrs,
            intermediate_values=trial.intermediate_values,
        )
        return new_trial

    return converter

In [ ]:
# add parameters (execute)
converter = add_distributions_params_map(add_distributions_params)

new_study = optuna.create_study()
new_study.add_trials(map(lambda t: converter(t), study.trials))

new_study.optimize(new_objective, n_trials=20, n_jobs=1)

In [ ]:
# add parameters and modify distributions in objective
def new_objective(trial):
    x = trial.suggest_int('x', 0, 10)
    y = trial.suggest_uniform('y', 4, 6)
    z = trial.suggest_categorical('z',[0])
    w = trial.suggest_discrete_uniform('w', -2.0, 2.0, 0.2)
    add_param1 = trial.suggest_uniform('add_param1', -2, 2)
    add_param2 = trial.suggest_categorical('add_param2',[False, True])
 
    # if suggest end, add set_suggest_end function
    trial.set_suggest_end()
 
    if add_param2 is True:
        return x ** 2 + (y - 5)** 2 + z + (w - 0.2) ** 2 + add_param1
    return x ** 2 + (y - 5)** 2 + z + (w - 0.2) ** 2
 
# add default value at new params
add_default_values = {
   'add_param1':0 ,
   'add_param2':False,
}
 

In [ ]:
#3. add parameters and modify distributions in objective (utility)
def new_objective_map(
    new_objective,  # type: ObjectiveFuncType
    add_default_values,  # type: Dict[str, List[Any]]
):
    # type: (...) -> Callable
    dummy_study = optuna.create_study()
    dummy_study.set_stop_opt()
    dummy_study.optimize(new_objective, n_trials=1, n_jobs=1)
    new_params = dummy_study.trials[0].params
    new_distributions = dummy_study.trials[0].distributions

    modify_distributions = {}
    modify_distributions_done = 0
    
    def get_modify_distributions(trial):
        # type: (FrozenTrial) -> None
        nonlocal new_params, new_distributions, modify_distributions, modify_distributions_done
        params = trial.params
        diff_params1 = set(params.keys()) - set(new_params.keys())
        if len(diff_params1) != 0:
            raise ValueError("Parameter {} is disappered in new objective".format(diff_params1))

        diff_params2 = set(new_params.keys()) - set(params.keys()) - set(add_default_values.keys())
        if len(diff_params2) != 0:
            raise ValueError(
                "Parameter {} added in new objective is not defined default value".format(
                    diff_params2
                )
            )
        modify_distributions = {param: new_distributions[param] for param in params.keys()}
        modify_distributions_done = 1
        
    def converter(trial):
        # type: (FrozenTrial) -> Optional[FrozenTrial]
        nonlocal new_params, new_distributions, modify_distributions, modify_distributions_done
        if modify_distributions_done == 0:
            get_modify_distributions(trial)

        if _check_param(trial.params, modify_distributions) is False:
            return None

        new_params = trial.params.copy()
        new_params.update(add_default_values)

        new_trial = optuna.trial.create_trial(
            state=trial.state,
            value=trial.value,
            params=new_params,
            distributions=new_distributions,
            user_attrs=trial.user_attrs,
            system_attrs=trial.system_attrs,
            intermediate_values=trial.intermediate_values,
        )
        return new_trial

    return converter

In [ ]:
# add parameters and modify distributions in objective (execute)
converter = new_objective_map(new_objective, add_default_values)
 
new_study = optuna.create_study()
# add trials
new_study.add_trials(map(lambda t: converter(t), study.trials))
 
new_study.optimize(new_objective, n_trials=20, n_jobs=1)

In [ ]:
#4. new metric
def objective(trial, metric_name):
    x = trial.suggest_uniform('x', -10 , 10)
    y = trial.suggest_uniform('y', 3, 30)

    metrics = {
        'plus': x + y,
        'minus': x - y
    }
    trial.set_user_attr('metrics', metrics)

    return metrics[metric_name]


# Run the first optimization that uses 'plus' metric.
study_a  = optuna.create_study()
study_a.optimize(lambda trial: objective(trial, 'plus'), n_trials=10)


In [ ]:
# new metric(utility)
def new_metric_map(
    metric_name,  # type: str
    metrics_attr="metrics",  # type: str
):
    # type: (...) -> Callable
    def converter(trial):
        # type: (FrozenTrial) -> FrozenTrial

        new_trial = optuna.trial.create_trial(
            state=trial.state,
            value=trial.user_attrs[metrics_attr][metric_name],
            params=trial.params,
            distributions=trial.distributions,
            user_attrs=trial.user_attrs,
            system_attrs=trial.system_attrs,
            intermediate_values=trial.intermediate_values,
        )
        return new_trial

    return converter

In [ ]:
# new metric(execute)
converter = new_metric_map(metric_name = 'minus')
study_b = optuna.study.create_study()
study_b.add_trials(map(lambda t: converter(t), study_a.trials))

assert len(study_b.trials) == 10

# Run the second optimization that uses 'minus' metric.
study_b.optimize(lambda trial: objective(trial, 'minus'), n_trials=10)
assert len(study_b.trials) == 20